In [103]:
from web3 import Web3
from web3.gas_strategies.rpc import rpc_gas_price_strategy
from uniswap import Uniswap
import requests
from dotenv import load_dotenv
import os
import json
import time
from datetime import datetime, timezone
import numpy as np

load_dotenv()

True

In [106]:
class TokenTrader:
    def __init__(self, rpc_url, etherscan_api_key, wallet_address, private_key, uniswap_contract_address, 
                 token_input_address, token_output_address, token_presale_contract_address, presale_id):
        self.rpc_url = rpc_url
        self.web3 = Web3(Web3.HTTPProvider(rpc_url))
        self.web3.eth.set_gas_price_strategy(rpc_gas_price_strategy)
        self.etherscan_api_key = etherscan_api_key
        self.wallet_address = wallet_address
        self.private_key = private_key
        # Init Web3 objects
        self.uniswap_contract_address = self.web3.to_checksum_address(uniswap_contract_address)
        self.uniswap_swap_contract_abi = self.get_contract_abi(self.uniswap_contract_address)
        self.uniswap_swap_contract_object = self.web3.eth.contract(address=self.uniswap_contract_address, abi=self.uniswap_swap_contract_abi)
        # Init Uniswap object
        self.uniswap = Uniswap(address=self.wallet_address, private_key=self.private_key, version=3, web3=self.web3, provider=self.rpc_url)
        # Presale contract
        self.token_presale_contract_address = self.web3.to_checksum_address(token_presale_contract_address)
        self.token_presale_contract_abi = self.get_contract_abi(self.token_presale_contract_address)
        self.token_presale_contract_object = self.web3.eth.contract(address=self.token_presale_contract_address, abi=self.token_presale_contract_abi)
        self.presale_id = presale_id
        # Init Token INPUT objects
        self.token_input_address = self.web3.to_checksum_address(token_input_address)
        self.token_input_abi = self.get_contract_abi(self.token_input_address)
        self.token_input_object = self.web3.eth.contract(address=self.token_input_address, abi=self.token_input_abi)
        self.token_input_symbol = self.token_input_object.functions.symbol().call()
        self.token_input_decimals = self.token_input_object.functions.decimals().call()
        # Init Token OUTPUT objects
        self.token_output_address = self.web3.to_checksum_address(token_output_address)
        self.token_output_abi = self.get_contract_abi(self.token_output_address)
        self.token_output_object = self.web3.eth.contract(address=self.token_output_address, abi=self.token_output_abi)
        self.token_output_symbol = self.token_output_object.functions.symbol().call()
        self.token_output_decimals = self.token_output_object.functions.decimals().call()
     
    def get_contract_abi(self, contract_address):
        try:
            url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={contract_address}&apikey={self.etherscan_api_key}"
            response = requests.get(url)
            response_json = response.json()
            # Comprueba si la solicitud fue exitosa y si hay un ABI disponible
            if response_json['status'] == '1' and response_json['message'] == 'OK':
                abi = json.loads(response_json['result'])
                return abi
        except Exception as e:
            print(f"Ha ocurrido un error al obtener el ABI: {e}")
            return None
        
    def is_claim_enabled(self):
        presale_details = self.token_presale_contract_object.functions.presale(self.presale_id).call()
        is_claim_enabled = presale_details[9]
        if not is_claim_enabled:
            return self.token_presale_contract_object.functions.userClaimData(self.wallet_address, self.presale_id).call()
        
    def get_user_claim_data(self):
        data = self.token_presale_contract_object.functions.userClaimData(self.wallet_address, self.presale_id).call()
        return {
            "investedAmount": data[0],
            "claimAt": data[1],
            "claimAbleAmount": data[2],
            "claimedVestingAmount": data[3],
            "claimedAmount": data[4],
            "claimCount": data[5],
            "activePercentAmount": data[6]
        }
    
    def claim_tokens(self):
        claim_data = self.get_user_claim_data()
        current_time = int(datetime.now(tz=timezone.utc).timestamp())
        if current_time >= claim_data['claimAt'] and claim_data['claimAbleAmount'] > 0:
            print(f"Intentando reclamar {claim_data['claimAbleAmount']/10**self.token_input_decimals} {self.token_input_symbol}.")
            # Preparar la transacción
            claim_txn = self.token_presale_contract_object.functions.claimAmount(self.presale_id).build_transaction({
                'chainId': self.web3.eth.chain_id,
                'gas': '0',
                'gasPrice': self.web3.eth.generate_gas_price(),
                'nonce': self.web3.eth.get_transaction_count(self.wallet_address),
            })
            gas = self.web3.eth.estimate_gas(claim_txn)
            claim_txn.update({'gas': gas})
            # Firmar la transacción
            signed_txn = self.web3.eth.account.sign_transaction(claim_txn, private_key=self.private_key)
            # Enviar la transacción
            tx_hash = self.web3.eth.send_raw_transaction(signed_txn.rawTransaction)
            print(f"CTransacción de reclamo enviada con hash: {tx_hash.hex()}")
            # Esperar a que la transacción sea confirmada
            tx_receipt = self.web3.eth.wait_for_transaction_receipt(tx_hash)
            if tx_receipt.status == 1:
                print(f"Claim successful! Transaction hash: {tx_hash.hex()}")
            else:
                print("Claim transaction failed.")

        else:
            print("No claimable tokens at the moment.")
        
    def get_price(self):
        price = self.uniswap.get_price_input(self.token_input_address, self.token_output_address, 10**self.token_input_decimals)
        return price / 10**self.token_output_decimals
    
    def swap(self, qty):
        print(f"Vendiendo {(qty)} {self.token_input_symbol} a {self.get_price()} {self.token_output_symbol}")
        qty = qty * (10 ** self.token_input_decimals)
        while True:
            try:
                tx = self.uniswap.make_trade(self.token_input_address, self.token_output_address, int(qty))
                print(f"Esperando por confirmación de la transacción {tx.hex()}")
                tx_receipt = self.web3.eth.wait_for_transaction_receipt(tx)
                if tx_receipt.status == 1:
                    print(f"Transacción confirmada: {tx.hex()}")
                break
            except Exception as e:
                print(f"Transacción fallida, reintentando... Error: {str(e)}\n")
                time.sleep(5)

In [107]:
# Configuración del proveedor
rpc_url = "https://rpc.ankr.com/eth"
etherscan_api_key = os.getenv("ETHERSCAN_API_KEY")
wallet_address = os.getenv("WALLET_ADDRESS")
private_key = os.getenv("PRIVATE_KEY")
uniswap_contract_address = "0x1458770554b8918B970444d8b2c02A47F6dF99A7"
token_input_address = "0x26EbB8213fb8D66156F1Af8908d43f7e3e367C1d"
token_output_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
token_presale_contract_address = "0x602C90D796D746b97a36f075d9f3b2892B9B07c2"
presale_id = 2
token_trader = TokenTrader(rpc_url,
                           etherscan_api_key,
                           wallet_address, 
                           private_key,
                           uniswap_contract_address,
                           token_input_address,
                           token_output_address,
                           token_presale_contract_address,
                           presale_id)
token_trader.approve_contract(1000)

ABIFunctionNotFound: ("The function 'approve' was not found in this contract's abi.", ' Are you sure you provided the correct contract abi?')